In [ ]:
!sudo apt install libimage-exiftool-perl -y

In [ ]:
import subprocess
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os

"""
    get XML metadata
"""

def get_xml_metadata(imgPath):
  infoDict = {}
  exifToolPath = 'exiftool'
  ''' use Exif tool to get the metadata '''
  process = subprocess.Popen([exifToolPath,imgPath],stdout=subprocess.PIPE, stderr=subprocess.STDOUT,universal_newlines=True)
  ''' get the tags in dict '''
  for tag in process.stdout:
      line = tag.strip().split(':')
      infoDict[line[0].strip()] = line[-1].strip()
  return infoDict

"""
    Vignette Correction
"""

def vig_correct(image_path, infoDict):
  centerX = int(float(infoDict['Calibrated Optical Center X']))
  centerY = int(float(infoDict['Calibrated Optical Center Y']))
  k = [float(elem.strip(",")) for elem in infoDict['Vignetting Data'].split()]

  # Load the image
  image = Image.open(image_path)
  np_img = np.array(image, dtype=np.uint16)
  rows, cols = np_img.shape[:2]

  # Create meshgrid of coordinates
  y_coords, x_coords = np.meshgrid(np.arange(rows), np.arange(cols), indexing='ij')

  # Calculate the distance r for each pixel
  r = np.sqrt((x_coords - centerX)**2 + (y_coords - centerY)**2)

  # Calculate the correction factor
  correction_factor = (
      k[5] * r**6 +
      k[4] * r**5 +
      k[3] * r**4 +
      k[2] * r**3 +
      k[1] * r**2 +
      k[0] * r +
      1.0
  )

  # Apply the correction factor to the image
  corrected_img = np_img * correction_factor[..., np.newaxis] if np_img.ndim == 3 else np_img * correction_factor
  return corrected_img.astype('uint16')

"""
    Distortion correction
"""
def undistort(new_img, infoDict):
  centerX = int(float(infoDict['Calibrated Optical Center X']))
  centerY = int(float(infoDict['Calibrated Optical Center Y']))

  dewarp_args = [float(elem) for elem in infoDict['Dewarp Data'].split(";")[1].split(",")]
  dist_coeffs = np.asarray(dewarp_args[4:])
  tuple0 = (dewarp_args[0], 0, centerX + dewarp_args[2])
  tuple1 = (0, dewarp_args[1], centerY + dewarp_args[3])
  tuple2 = (0, 0, 1)
  camera_matrix = np.asarray([tuple0, tuple1, tuple2])

  h,  w = new_img.shape[:2]
  _, roi=cv2.getOptimalNewCameraMatrix(camera_matrix,dist_coeffs,(w,h),1,(w,h))

  dst = cv2.undistort(new_img, camera_matrix, dist_coeffs, None, camera_matrix)
  # crop the image
  x,y,w,h = roi
  dst = dst[y:y+h, x:x+w]
  return dst


"""
    Alignment of the phase and rotation differences caused by different camera locations and
    optical accuracy
"""
def align_phase_rotation(new_img, infoDict):
  rows, cols = new_img.shape
  H = np.asarray([float(elem) for elem in infoDict['Calibrated H Matrix'].split(",")]).reshape(3,3)
  new_img = cv2.warpPerspective(new_img, H, (cols, rows))
  return new_img


In [ ]:
""""

Alignment of the difference caused by different exposure times

1. Smoothing the images using a Gaussian filter

2. Apply an edge detection filter (ex. Sobel filter) to detect edge lines from the
two images that need to be aligned.

3. Apply an alignment algorithm such as the
Enhanced Correlation Coefficient (ECC) Maximization to the images.

"""

from scipy.ndimage import gaussian_filter

def smooth_image(image, sigma=1):
    """Apply Gaussian smoothing to the image."""
    return gaussian_filter(image, sigma=sigma)

def edge_detection(image):
    """Apply Sobel filter to detect edges in the image."""
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    magnitude = cv2.magnitude(grad_x, grad_y)
    return magnitude



def crop_center(img, crop_tamanho):
    # Verificar as dimensões da imagem
  altura = img.shape[0]
  largura = img.shape[1]

  # Calcular as coordenadas do crop central
  start_x = largura // 2 - crop_tamanho // 2
  start_y = altura // 2 - crop_tamanho // 2
  end_x = start_x + crop_tamanho
  end_y = start_y + crop_tamanho

  # Realizar o crop central
  imagem_cropped = img[start_y:end_y, start_x:end_x]
  return imagem_cropped

def zoom_center(img, zoom_factor=1.5):

    y_size = img.shape[0]
    x_size = img.shape[1]

    # define new boundaries
    x1 = int(0.5*x_size*(1-1/zoom_factor))
    x2 = int(x_size-0.5*x_size*(1-1/zoom_factor))
    y1 = int(0.5*y_size*(1-1/zoom_factor))
    y2 = int(y_size-0.5*y_size*(1-1/zoom_factor))

    # first crop image then scale
    img_cropped = img[y1:y2,x1:x2]
    return cv2.resize(img_cropped, None, fx=zoom_factor, fy=zoom_factor)

IMG_REF_SHAPE = (2570, 1925)
def process_image(imgPath):
  ##### check image 'Image Source': 'MS_G_CAMERA' for camera type

  # get xml metadata for camera corrections
  infoDict = get_xml_metadata(imgPath)

  # custom pipeline for jpg images, because they have a different resolution 
  if imgPath[-3:] == 'JPG':
        new_img = cv2.imread(imgPath)
        new_img = zoom_center(new_img, 1.3)
        new_img = cv2.resize(new_img, IMG_REF_SHAPE)
        new_img = crop_center(new_img, 1500)
        return new_img


  # apply vignette correction
  new_img = vig_correct(imgPath, infoDict)

  # undistort image
  new_img = undistort(new_img, infoDict)

  # align phase and rotation
  new_img = align_phase_rotation(new_img, infoDict)
  
  # crop center
  new_img = crop_center(new_img, 1500)

  return new_img

In [ ]:
"""

    Apply alignment algorithm!

"""
def align_images_using_ecc(reference_image, target_image, jpg=False):
    if jpg:
        # save colored jpg
        colored_jpg = target_image
        # create temporary greyscale img
        target_image = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY) 
    
    # Smooth the images
    reference_image_smoothed = smooth_image(reference_image)
    target_image_smoothed = smooth_image(target_image)

    # Apply edge detection
    base_edges = edge_detection(reference_image_smoothed)
    target_edges = edge_detection(target_image_smoothed)

    # Convert images to float32 for ECC algorithm
    base_edges = base_edges.astype(np.float32)
    target_edges = target_edges.astype(np.float32)
    
    # Define the motion model
    warp_mode = cv2.MOTION_HOMOGRAPHY
    
    # Define 3x3 transformation matrix
    warp_matrix = np.eye(3, 3, dtype=np.float32)

    # Set the number of iterations and termination criteria
    if jpg:
        number_of_iterations = 500
    else:
        number_of_iterations = 25
    termination_eps = 1e-10

    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations, termination_eps)


    # Apply the ECC algorithm to find the warp matrix
    cc, warp_matrix = cv2.findTransformECC(base_edges, target_edges, warp_matrix, warp_mode, criteria)
    
    if jpg:
        target_image = colored_jpg

    # Warp the target image to align with the base image
    aligned_image = cv2.warpPerspective(target_image, warp_matrix, (reference_image.shape[1], reference_image.shape[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    
    return aligned_image

In [ ]:
# DRIVER
import math
from itertools import cycle , islice

# Take relative paths, because chenged os current dir
os.chdir("/kaggle/input/27-01-25-voo-3/DJI_202501271001_009_Linear-Flight-Mission2-copy")
types = ('*.JPG', '*.TIF') # the tuple of file types
files_grabbed = []
for files in types:
    files_grabbed.extend(glob.glob(files))
    
input_imgs = sorted(files_grabbed)[1030:2060]

i = cycle(input_imgs)
slc = 5
for _ in range(math.ceil(len(input_imgs)/slc)):
    cur_imgs = list(islice(i,slc))
    
    try:

        ref_img = process_image(cur_imgs[1]) # G BAND -> REFERENCE
        target_jpg = process_image(cur_imgs[0])
        target_nir = process_image(cur_imgs[2])
        target_r = process_image(cur_imgs[3])
        target_re = process_image(cur_imgs[4])

        # Align image to G BAND
        aligned_jpg_image = align_images_using_ecc(ref_img, target_jpg, True)
        aligned_nir_image = align_images_using_ecc(ref_img, target_nir)
        aligned_r_image = align_images_using_ecc(ref_img, target_r)
        aligned_re_image = align_images_using_ecc(ref_img, target_re)

        # Remember to crop and normalize reference/target img here:
        # Normalize the image to range 0 to 255 and convert to uint8
        aligned_jpg_image = cv2.normalize(aligned_jpg_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        aligned_nir_image = cv2.normalize(aligned_nir_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        aligned_r_image = cv2.normalize(aligned_r_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        aligned_re_image = cv2.normalize(aligned_re_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        aligned_g_image = cv2.normalize(ref_img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

        # crop again
        aligned_jpg_image = crop_center(aligned_jpg_image, 1000)
        aligned_nir_image = crop_center(aligned_nir_image, 1000)
        aligned_r_image = crop_center(aligned_r_image, 1000)
        aligned_re_image = crop_center(aligned_re_image, 1000)
        aligned_g_image = crop_center(aligned_g_image, 1000)

        # save
        cv2.imwrite(f"/kaggle/working/processed-{cur_imgs[0]}", aligned_jpg_image)
        cv2.imwrite(f"/kaggle/working/processed-{cur_imgs[1]}", aligned_g_image)
        cv2.imwrite(f"/kaggle/working/processed-{cur_imgs[2]}", aligned_nir_image)
        cv2.imwrite(f"/kaggle/working/processed-{cur_imgs[3]}", aligned_r_image)
        cv2.imwrite(f"/kaggle/working/processed-{cur_imgs[4]}", aligned_re_image)
    except:
        print(f"erro no batch: {cur_imgs}")
